# Cell Instance Segmentation Unet

This notebook referred to the following notebook.<br/>
https://www.kaggle.com/arunamenon/cell-instance-segmentation-unet-eda<br/>
https://www.kaggle.com/karan23258/cell-instance-segmentation-unetfromscratch<br/>
https://www.kaggle.com/evangelou/sartorius-unet-pytorch-from-scratch

# Import packages

In [ ]:
import numpy as np
import pandas as pd
import imageio
import matplotlib.pyplot as plt
import cv2
from skimage.io import imread, imshow, imread_collection, concatenate_images
from skimage.transform import resize
from tqdm import tqdm
from keras.models import Model, load_model
from keras.layers import Input
from keras.layers.core import Dropout, Lambda
from keras.layers.convolutional import Conv2D, Conv2DTranspose
from keras.layers.pooling import MaxPooling2D
from keras.layers.merge import concatenate
from keras.callbacks import EarlyStopping, ModelCheckpoint
from keras import backend as K
import tensorflow as tf
from skimage.morphology import label
import random

# Load input files

In [ ]:
train_data = pd.read_csv('../input/sartorius-cell-instance-segmentation/train.csv')

In [ ]:
print(train_data.shape)
train_data.head()

In [ ]:
print(train_data['cell_type'].unique().tolist())

In [ ]:
print(train_data[train_data['cell_type']=='shsy5y']['id'].tolist()[0])
print(train_data[train_data['cell_type']=='astro']['id'].tolist()[0])
print(train_data[train_data['cell_type']=='cort']['id'].tolist()[0])

## rle_decode

In [ ]:
# Reference: https://www.kaggle.com/ihelon/cell-segmentation-run-length-decoding

def rle_decode(mask_rle, shape, color=3):     #color=1,3
    '''
    mask_rle: run-length as string formated (start length)
    shape: (height, width, channels) of array to return 
    color: color for the mask
    Returns numpy array (mask)
    '''
    s = mask_rle.split()
    
    starts = list(map(lambda x: int(x) - 1, s[0::2]))
    lengths = list(map(int, s[1::2]))
    ends = [x + y for x, y in zip(starts, lengths)]
    
    img = np.zeros((shape[0] * shape[1], shape[2]), dtype=np.float32)
            
    for start, end in zip(starts, ends):
        img[start : end] = color
    
    return img.reshape(shape)


## plot_masks

In [ ]:
def plot_masks(image_id, colors=True):
    labels = train_data[train_data["id"] == image_id]["annotation"].tolist()

    if colors:
        mask = np.zeros((520, 704, 3))
        for label in labels:
            mask += rle_decode(label, shape=(520, 704, 3), color=np.random.rand(3))
    else:
        mask = np.zeros((520, 704, 1))
        for label in labels:
            mask += rle_decode(label, shape=(520, 704, 1))
    mask = mask.clip(0, 1)

    image = cv2.imread(f"../input/sartorius-cell-instance-segmentation/train/{image_id}.png")
    image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)

    plt.figure(figsize=(18,6))
    plt.subplot(1, 3, 1)
    plt.imshow(image)
    plt.title('Input image')
    plt.axis("off")
    
    plt.subplot(1, 3, 2)
    plt.imshow(image)
    plt.imshow(mask, alpha=0.1)
    plt.title('Input image with mask')
    plt.axis("off")
    
    plt.subplot(1, 3, 3)
    plt.imshow(mask)
    plt.title('Only mask')
    plt.axis("off")
    
    plt.show();

In [ ]:
sample_ids = ['0030fd0e6378','0140b3c8f445','01ae5a43a2ab']

for sample_id in sample_ids:
    celltype=train_data[train_data['id']==sample_id]['cell_type'].tolist()[0]
    file_path = '../input/sartorius-cell-instance-segmentation/train/' + sample_id + '.png'
    image_df = imageio.imread(file_path)
    print('ID:', sample_id, ', CellType:',celltype)
    plot_masks(sample_id, colors=False)

# Modelling

In [ ]:
sample_submission=pd.read_csv('../input/sartorius-cell-instance-segmentation/sample_submission.csv')

In [ ]:
# Reference: https://www.kaggle.com/keegil/keras-u-net-starter-lb-0-277
IMG_HEIGHT = 256
IMG_WIDTH = 256
IMG_CHANNELS = 1
TRAIN_PATH = '../input/sartorius-cell-instance-segmentation/train/'

train_ids = train_data['id'].unique().tolist()
test_ids = sample_submission['id'].unique().tolist()

# Get and resize train images and masks
X_train = np.zeros((train_data['id'].nunique(), IMG_HEIGHT, IMG_WIDTH, IMG_CHANNELS), dtype=np.uint8)
Y_train = np.zeros((train_data['id'].nunique(), IMG_HEIGHT, IMG_WIDTH, 1), dtype=np.bool)

In [ ]:
for n, id_ in tqdm(enumerate(train_ids), total=len(train_ids)):
    path = TRAIN_PATH + id_
    img = imread(path + '.png')[:,:]
    img = resize(img, (IMG_HEIGHT, IMG_WIDTH), mode='constant', preserve_range=True)
    img = np.expand_dims(img, axis = 2)
    X_train[n] = img
    
    labels = train_data[train_data["id"] == id_]["annotation"].tolist()
    mask = np.zeros((520, 704, 1))
    for label in labels:
        mask += rle_decode(label, shape=(520, 704, 1), color = 1)
    mask = mask.clip(0, 1)
    mask = mask[:,:,0]

    mask = np.expand_dims(resize(mask, (IMG_HEIGHT, IMG_WIDTH), mode='constant', 
                                  preserve_range=True), axis=-1)
    
    Y_train[n] = mask

In [ ]:
# Get and resize test images
X_test = np.zeros((sample_submission['id'].nunique(), IMG_HEIGHT, IMG_WIDTH, IMG_CHANNELS), dtype=np.uint8)

for n, id_ in tqdm(enumerate(test_ids), total=len(test_ids)):
    path = TRAIN_PATH.replace('train', 'test') + id_
    img = imread(path + '.png')[:,:]
    img = resize(img, (IMG_HEIGHT, IMG_WIDTH), mode='constant', preserve_range=True)
    img = np.expand_dims(img, axis = 2)
    X_test[n] = img

In [ ]:
print(X_train.shape,Y_train.shape,X_test.shape)

In [ ]:
sample_id_num = 40
plt.imshow(X_train[sample_id_num][:,:,0], cmap = 'gray')
plt.show()
plt.imshow(Y_train[sample_id_num][:,:,0])
plt.show()

print('Input image:','Min:', X_train[sample_id_num][:,:,0].min(), '; Max:', X_train[sample_id_num][:,:,0].max(), '; Mean:', X_train[sample_id_num][:,:,0].mean())
print('Mask:','Min:', Y_train[sample_id_num][:,:,0].min(), '; Max:', Y_train[sample_id_num][:,:,0].max(), '; Mean:', Y_train[sample_id_num][:,:,0].mean())

In [ ]:
tf.to_int32 = lambda x: tf.cast(x, tf.int32)

def mean_iou(y_true, y_pred, smooth=1):
    intersection = K.sum(K.abs(y_true * y_pred), axis=[1,2,3])
    union = K.sum(y_true,[1,2,3])+K.sum(y_pred,[1,2,3])-intersection
    iou = K.mean((intersection + smooth) / (union + smooth), axis=0)
    return tf.convert_to_tensor(iou)

## dice_coefficient

In [ ]:
def dice_coefficient(y_true, y_pred):
    numerator = 2 * tf.reduce_sum(y_true * y_pred)
    denominator = tf.reduce_sum(y_true + y_pred)
    return numerator / (denominator + tf.keras.backend.epsilon())

### Reference: 'U-Net: Convolutional Networks for Biomedical Image Segmentation'
https://arxiv.org/pdf/1505.04597.pdf

In [ ]:
# Build U-Net model
inputs = Input((IMG_HEIGHT, IMG_WIDTH, IMG_CHANNELS))
s = Lambda(lambda x: x / 255) (inputs)

c1 = Conv2D(16, (3, 3), activation='elu', kernel_initializer='he_normal', padding='same') (s)
c1 = Dropout(0.1) (c1)
c1 = Conv2D(16, (3, 3), activation='elu', kernel_initializer='he_normal', padding='same') (c1)
p1 = MaxPooling2D((2, 2)) (c1)

c2 = Conv2D(32, (3, 3), activation='elu', kernel_initializer='he_normal', padding='same') (p1)
c2 = Dropout(0.1) (c2)
c2 = Conv2D(32, (3, 3), activation='elu', kernel_initializer='he_normal', padding='same') (c2)
p2 = MaxPooling2D((2, 2)) (c2)

c3 = Conv2D(64, (3, 3), activation='elu', kernel_initializer='he_normal', padding='same') (p2)
c3 = Dropout(0.2) (c3)
c3 = Conv2D(64, (3, 3), activation='elu', kernel_initializer='he_normal', padding='same') (c3)
p3 = MaxPooling2D((2, 2)) (c3)

c4 = Conv2D(128, (3, 3), activation='elu', kernel_initializer='he_normal', padding='same') (p3)
c4 = Dropout(0.2) (c4)
c4 = Conv2D(128, (3, 3), activation='elu', kernel_initializer='he_normal', padding='same') (c4)
p4 = MaxPooling2D(pool_size=(2, 2)) (c4)

c5 = Conv2D(256, (3, 3), activation='elu', kernel_initializer='he_normal', padding='same') (p4)
c5 = Dropout(0.3) (c5)
c5 = Conv2D(256, (3, 3), activation='elu', kernel_initializer='he_normal', padding='same') (c5)

u6 = Conv2DTranspose(128, (2, 2), strides=(2, 2), padding='same') (c5)
u6 = concatenate([u6, c4])
c6 = Conv2D(128, (3, 3), activation='elu', kernel_initializer='he_normal', padding='same') (u6)
c6 = Dropout(0.2) (c6)
c6 = Conv2D(128, (3, 3), activation='elu', kernel_initializer='he_normal', padding='same') (c6)

u7 = Conv2DTranspose(64, (2, 2), strides=(2, 2), padding='same') (c6)
u7 = concatenate([u7, c3])
c7 = Conv2D(64, (3, 3), activation='elu', kernel_initializer='he_normal', padding='same') (u7)
c7 = Dropout(0.2) (c7)
c7 = Conv2D(64, (3, 3), activation='elu', kernel_initializer='he_normal', padding='same') (c7)

u8 = Conv2DTranspose(32, (2, 2), strides=(2, 2), padding='same') (c7)
u8 = concatenate([u8, c2])
c8 = Conv2D(32, (3, 3), activation='elu', kernel_initializer='he_normal', padding='same') (u8)
c8 = Dropout(0.1) (c8)
c8 = Conv2D(32, (3, 3), activation='elu', kernel_initializer='he_normal', padding='same') (c8)

u9 = Conv2DTranspose(16, (2, 2), strides=(2, 2), padding='same') (c8)
u9 = concatenate([u9, c1], axis=3)
c9 = Conv2D(16, (3, 3), activation='elu', kernel_initializer='he_normal', padding='same') (u9)
c9 = Dropout(0.1) (c9)
c9 = Conv2D(16, (3, 3), activation='elu', kernel_initializer='he_normal', padding='same') (c9)

outputs = Conv2D(1, (1, 1), activation='sigmoid') (c9)

model = Model(inputs=[inputs], outputs=[outputs])
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=[dice_coefficient])
#model.summary()

In [ ]:
# Fit model
earlystopper = EarlyStopping(patience=20, verbose=1)
checkpointer = ModelCheckpoint('best_model.h5', verbose=1, save_best_only=True)
results = model.fit(X_train, Y_train, validation_split=0.15, batch_size=5, epochs=50, 
                    callbacks=[earlystopper, checkpointer])

In [ ]:
plt.figure(figsize=(12,4))
plt.plot(results.history['loss'])
plt.plot(results.history['val_loss'])
plt.title('model loss')
plt.ylabel('Loss')
plt.xlabel('epoch')
plt.legend(['train', 'val'], loc='upper right')
plt.show()

In [ ]:
plt.figure(figsize=(12,4))
plt.plot(results.history['dice_coefficient'])
plt.plot(results.history['val_dice_coefficient'])
plt.title('dice_coefficient')
plt.ylabel('dice_coefficient')
plt.xlabel('epoch')
plt.legend(['train', 'val'], loc='upper left')
plt.show()

# Prediction

In [ ]:
# Predict on train, val and test
model = load_model('best_model.h5', custom_objects={'dice_coefficient': dice_coefficient})
preds_train = model.predict(X_train[:int(X_train.shape[0]*0.9)], verbose=1)
preds_val = model.predict(X_train[int(X_train.shape[0]*0.9):], verbose=1)
preds_test = model.predict(X_test, verbose=1)

# Threshold predictions
preds_train_t = (preds_train > 0.5).astype(np.uint8)
preds_val_t = (preds_val > 0.5).astype(np.uint8)
preds_test_t = (preds_test > 0.5).astype(np.uint8)

# Create list of upsampled test masks
preds_test_upsampled = []
for i in range(len(preds_test)):
    preds_test_upsampled.append(resize(np.squeeze(preds_test[i]), 
                                    (IMG_HEIGHT, IMG_WIDTH), 
                                    mode='constant', preserve_range=True))

In [ ]:
# Perform a sanity check on some random training samples
ix = random.randint(0, len(preds_train_t))
imshow(X_train[ix])
plt.show()
imshow(np.squeeze(Y_train[ix]))
plt.show()
imshow(np.squeeze(preds_train_t[ix]))
plt.show()

In [ ]:
# Perform a sanity check on some random validation samples
ix = random.randint(0, len(preds_val_t))
imshow(X_train[int(X_train.shape[0]*0.9):][ix])
plt.show()
imshow(np.squeeze(Y_train[int(Y_train.shape[0]*0.9):][ix]))
plt.show()
imshow(np.squeeze(preds_val_t[ix]))
plt.show()

In [ ]:
# Test samples
ix = random.randint(0, len(preds_test_t)-1)
imshow(X_test[ix])
plt.show()
imshow(np.squeeze(preds_test_t[ix]))
plt.show()

## rle_encoding

In [ ]:
def rle_encoding(x):
    dots = np.where(x.flatten() == 1)[0]
    run_lengths = []
    prev = -2
    for b in dots:
        if (b>prev+1): run_lengths.extend((b + 1, 0))
        run_lengths[-1] += 1
        prev = b
    return ' '.join(map(str, run_lengths))

In [ ]:
for pred in preds_test_t:
    plt.imshow(pred)
    plt.title('pred before resize')
    plt.axis("off")
    plt.show()

In [ ]:
#test_mask: after reshape before fix_overlapping
for pred in preds_test_t:
    print(pred.shape)
    test_mask=cv2.resize(pred,dsize=(704,520),interpolation=cv2.INTER_CUBIC).reshape(520,704,1)
    print(test_mask.shape)
        
    plt.imshow(test_mask)
    plt.title('pred after resize')
    plt.axis("off")
    plt.show()

In [ ]:
test_masks = [cv2.resize(pred,dsize=(704,520),interpolation=cv2.INTER_CUBIC).reshape(520,704,1) for pred in preds_test_t]
print(test_masks[0].shape)
print(test_masks[1].shape)
print(test_masks[2].shape)

## fix_overlap
https://www.kaggle.com/c/sartorius-cell-instance-segmentation/discussion/279995

In [ ]:
def check_overlap(msk):
    msk = msk.astype(np.bool).astype(np.uint8)
    return np.any(np.sum(msk, axis=-1)>1)

In [ ]:
def fix_overlap(msk):
    """
    Args:
        mask: multi-channel mask, each channel is an instance of cell, shape:(520,704,None)
    Returns:
        multi-channel mask with non-overlapping values, shape:(520,704,None)
    """
    msk = np.array(msk)
    msk = np.pad(msk, [[0,0],[0,0],[1,0]])
    ins_len = msk.shape[-1]
    msk = np.argmax(msk,axis=-1)
    msk = tf.keras.utils.to_categorical(msk, num_classes=ins_len)
    msk = msk[...,1:]
    msk = msk[...,np.any(msk, axis=(0,1))]
    return msk

#### When prediction resulted in failure, error message said ValueError: cannot reshape array of size 0 into shape (520,704,1)

In [ ]:
for test_mask in test_masks:
    overlap_test_masks=check_overlap(test_mask)
    print(overlap_test_masks)

In [ ]:
#test_mask2: after reshape after fix_overlapping
test_masks2=[]
for test_mask in test_masks:
    test_mask2 = fix_overlap(test_mask).reshape(520,704,1)
    print(test_mask2.shape)
    test_masks2+=[test_mask2]

In [ ]:
for test_mask2 in test_masks2:
    overlap_test_masks2=check_overlap(test_mask2)
    print(overlap_test_masks2)

predicted = [rle_encoding(test_mask) for test_mask in test_masks]
print(predicted[0])

In [ ]:
predicted2 = [rle_encoding(test_mask2) for test_mask2 in test_masks2]
#print(predicted2[0])

In [ ]:
submit = sample_submission.copy()
submit['predicted'] = predicted2
submit.to_csv('submission.csv', index=False)
submit

In [ ]:
ids=sample_submission['id'].tolist()
print(ids)
submit2=pd.DataFrame(columns=sample_submission.columns)  

for idi, test_mask2 in zip(ids,test_masks2):
    annos=rle_encoding(test_mask2)
    annos2=annos.split(' ')
    annos4=[]
    for i in range(len(annos2)//100):
        annos3=''
        for j in range(100):
            annos3+=annos2[i*100+j]+' '
        annos4+=[annos3]
        
    submit=pd.DataFrame(columns=sample_submission.columns)    
    submit['predicted']=annos4
    submit['id']=idi
    submit2=pd.concat([submit2,submit],axis=0)
    
#submit2.to_csv('submission.csv', index=False)
submit2

In [ ]:
plt.imshow(test_masks[0])
plt.title('before fix_overlapping')
plt.axis("off")
plt.show()

plt.imshow(test_masks2[0])
plt.title('after fix_overlapping')
plt.axis("off")
plt.show()